In [1]:
import os
import camelot
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
dict_fetch = {"Past-Due and Nonaccrual Loans / Total Loans" : "Asset Quality",
              "Noncurrent Loans / Total Loans" : "Asset Quality",
              "Loan and Lease Allowance / Total Loans" : "Asset Quality",
              "Loan and Lease Allowance / Noncurrent Loans" : "Asset Quality",
              "Net Loan Losses / Total Loans" : "Asset Quality",
              "Return on Assets" : "Earnings",
              "Net Interest Margin" : "Earnings",
              "Yield on Earning Assets" : "Earnings",
              "Cost of Funding Earning Assets" : "Earnings",
              "Provisions to Avg. Assets" : "Earnings",
              "Overhead to Avg. Assets" : "Earnings",
              "Net Loans to Assets" : "Liquidity",
              "Long-term Assets to Assets" : "Liquidity"
            }

In [4]:
files_path = '/home/shubha/work/byteiq_tasks/fdic/download/'

In [5]:
lf_paths = []

In [8]:
download_list = os.listdir(files_path)
download_list.sort()

In [6]:
def extract(file_path):    
    try:
        tables = camelot.read_pdf(filepath=file_path,flavor='stream',edge_tol=1500)
        temp_df = tables[1].df
        remove_from_bracket = lambda text : text.split('(', 1)[0]
        remove_spaces = lambda text : text.strip()
        temp_df[0] = temp_df[0].map(remove_from_bracket).map(remove_spaces)
        temp_df = temp_df[0:52]
        temp_df = temp_df.drop([1], axis=1)
        column_name_1 = temp_df[0][1]
        temp_df = temp_df.replace(r'^\s*$', np.nan, regex=True)
        temp_df = temp_df.dropna()
        temp_df = temp_df.reset_index(drop = True)
        temp_df.columns = temp_df.iloc[0]
        temp_df = temp_df.drop(temp_df.index[0])
        temp_df.rename(columns={'Employment Growth Rates': column_name_1}, inplace=True)
        temp_df = temp_df.reset_index(drop = True)
        filter_df = pd.DataFrame()
        filter_df['Category'] = ""
        for i in range(len(temp_df)):
            eco_indi = temp_df['ECONOMIC INDICATORS'][i]
            for key,value in dict_fetch.items():
                if key == eco_indi:
                    filter_df = filter_df.append(temp_df[temp_df['ECONOMIC INDICATORS'] == eco_indi])
                    filter_df['Category'][i] = value
        filter_df = filter_df[['ECONOMIC INDICATORS', 'Q2-21', 'Q1-21', 'Q2-20', '2020', '2019', 'Category']]
        filter_df = filter_df.reset_index(drop = True)
        filter_df['state_alias'] = (file_path.split('/')[-1]).split('.')[0]
        return filter_df
    except:
        lf_paths.append(file_path)

In [9]:
file_path = ''
final_df = pd.DataFrame()
for file in download_list:
    file_path = files_path + file
    temp_df = extract(file_path)
    final_df = final_df.append(temp_df)
final_df = final_df.reset_index(drop = True)

In [10]:
lf_paths

['/home/shubha/work/byteiq_tasks/fdic/download/ak.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/ca.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/de.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/hi.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/ms.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/nc.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/ne.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/nh.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/nm.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/ny.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/oh.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/sd.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/tx.pdf',
 '/home/shubha/work/byteiq_tasks/fdic/download/wy.pdf']

In [27]:
def extract_handle_1(failed_path):
    tables = camelot.read_pdf(filepath=failed_path,flavor='stream',edge_tol=1500)
    temp_df = tables[1].df
    remove_from_bracket = lambda text : text.split('(', 1)[0]
    remove_spaces = lambda text : text.strip()
    temp_df[0] = temp_df[0].map(remove_from_bracket).map(remove_spaces)
    # temp_df = temp_df.drop([1], axis=1)
    temp_df = temp_df.replace(r'^\s*$', np.nan, regex=True)
    temp_df = temp_df.dropna()
    temp_df = temp_df.reset_index(drop = True)
    temp_df.columns = temp_df.iloc[0]
    temp_df = temp_df.drop(temp_df.index[0])
    temp_df.rename(columns={'Employment Growth Rates': 'ECONOMIC INDICATORS'}, inplace=True)
    temp_df = temp_df.reset_index(drop = True)
    filter_df = pd.DataFrame()
    filter_df['Category'] = ""
    for i in range(len(temp_df)):
        eco_indi = temp_df['ECONOMIC INDICATORS'][i]
        for key,value in dict_fetch.items():
            if key == eco_indi:
                filter_df = filter_df.append(temp_df[temp_df['ECONOMIC INDICATORS'] == eco_indi])
                filter_df['Category'][i] = value
    filter_df = filter_df[['ECONOMIC INDICATORS', 'Q2-21', 'Q1-21', 'Q2-20', '2020', '2019', 'Category']]
    filter_df = filter_df.reset_index(drop = True)
    filter_df['state_alias'] = (failed_path.split('/')[-1]).split('.')[0]
    return filter_df

In [28]:
def extract_handle_2(failed_path):
    tables = camelot.read_pdf(filepath=failed_path,flavor='stream',edge_tol=1500)
    temp_df = tables[0].df
    remove_from_bracket = lambda text : text.split('(', 1)[0]
    remove_spaces = lambda text : text.strip()
    temp_df[0] = temp_df[0].map(remove_from_bracket).map(remove_spaces)
    temp_df = temp_df.drop([1], axis=1)
    temp_df = temp_df.replace(r'^\s*$', np.nan, regex=True)
    temp_df = temp_df.dropna()
    temp_df = temp_df.reset_index(drop = True)
    temp_df.columns = temp_df.iloc[0]
    temp_df = temp_df.drop(temp_df.index[0])
    temp_df.rename(columns={'Employment Growth Rates': 'ECONOMIC INDICATORS'}, inplace=True)
    temp_df = temp_df.reset_index(drop = True)
    filter_df = pd.DataFrame()
    filter_df['Category'] = ""
    for i in range(len(temp_df)):
        eco_indi = temp_df['ECONOMIC INDICATORS'][i]
        for key,value in dict_fetch.items():
            if key == eco_indi:
                filter_df = filter_df.append(temp_df[temp_df['ECONOMIC INDICATORS'] == eco_indi])
                filter_df['Category'][i] = value
    filter_df = filter_df[['ECONOMIC INDICATORS', 'Q2-21', 'Q1-21', 'Q2-20', '2020', '2019', 'Category']]
    filter_df = filter_df.reset_index(drop = True)
    filter_df['state_alias'] = (failed_path.split('/')[-1]).split('.')[0]
    return filter_df

In [29]:
try_success_df = pd.DataFrame()

In [30]:
for failed_path in lf_paths:
    try:
        try_df = extract_handle_1(failed_path)
        print("Success")
    except:
        try_df = extract_handle_2(failed_path)
        print("Success Except")
    try_success_df = try_success_df.append(try_df)
try_success_df = try_success_df.reset_index(drop = True)

Success Except
Success
Success
Success Except
Success Except
Success
Success Except
Success Except
Success Except
Success
Success
Success
Success
Success Except


In [31]:
try_success_df

,ECONOMIC INDICATORS,Q2-21,Q1-21,Q2-20,2020,2019,Category,state_alias
0,Past-Due and Nonaccrual Loans / Total Loans,0.84,0.92,1.04,0.78,1.42,Asset Quality,ak
1,Noncurrent Loans / Total Loans,0.62,0.76,0.70,0.72,1.07,Asset Quality,ak
2,Loan and Lease Allowance / Total Loans,1.69,1.59,1.49,1.65,1.69,Asset Quality,ak
3,Loan and Lease Allowance / Noncurrent Loans,2.39,1.70,1.61,1.83,1.75,Asset Quality,ak
4,Net Loan Losses / Total Loans,0.00,-0.01,0.00,0.01,0.01,Asset Quality,ak
5,Return on Assets,1.30,1.41,0.91,1.02,1.11,Earnings,ak
6,Net Interest Margin,2.96,3.13,3.64,3.43,3.96,Earnings,ak
7,Yield on Earning Assets,3.12,3.16,3.79,3.52,4.19,Earnings,ak
8,Cost of Funding Earning Assets,0.16,0.17,0.28,0.26,0.39,Earnings,ak
9,Provisions to Avg. Assets,0.08,0.14,0.09,0.12,0.03,Earnings,ak


In [32]:
final_df

,ECONOMIC INDICATORS,Q2-21,Q1-21,Q2-20,2020,2019,Category,state_alias
0,Past-Due and Nonaccrual Loans / Total Loans,0.80,1.00,1.00,1.11,1.63,Asset Quality,al
1,Noncurrent Loans / Total Loans,0.43,0.44,0.49,0.48,0.66,Asset Quality,al
2,Loan and Lease Allowance / Total Loans,1.33,1.34,1.23,1.31,1.23,Asset Quality,al
3,Loan and Lease Allowance / Noncurrent Loans,2.94,2.92,2.17,2.27,1.80,Asset Quality,al
4,Net Loan Losses / Total Loans,0.02,0.00,0.04,0.07,0.12,Asset Quality,al
5,Return on Assets,1.03,0.97,0.98,0.93,1.07,Earnings,al
6,Net Interest Margin,3.28,3.21,3.70,3.56,3.86,Earnings,al
7,Yield on Earning Assets,3.69,3.65,4.37,4.18,4.71,Earnings,al
8,Cost of Funding Earning Assets,0.34,0.37,0.68,0.58,0.77,Earnings,al
9,Provisions to Avg. Assets,0.03,0.00,0.13,0.13,0.11,Earnings,al


In [34]:
absolute_df = pd.DataFrame()

In [35]:
absolute_df = absolute_df.append(final_df)

In [37]:
absolute_df = absolute_df.append(try_success_df)

In [38]:
absolute_df

,ECONOMIC INDICATORS,Q2-21,Q1-21,Q2-20,2020,2019,Category,state_alias
0,Past-Due and Nonaccrual Loans / Total Loans,0.80,1.00,1.00,1.11,1.63,Asset Quality,al
1,Noncurrent Loans / Total Loans,0.43,0.44,0.49,0.48,0.66,Asset Quality,al
2,Loan and Lease Allowance / Total Loans,1.33,1.34,1.23,1.31,1.23,Asset Quality,al
3,Loan and Lease Allowance / Noncurrent Loans,2.94,2.92,2.17,2.27,1.80,Asset Quality,al
4,Net Loan Losses / Total Loans,0.02,0.00,0.04,0.07,0.12,Asset Quality,al
5,Return on Assets,1.03,0.97,0.98,0.93,1.07,Earnings,al
6,Net Interest Margin,3.28,3.21,3.70,3.56,3.86,Earnings,al
7,Yield on Earning Assets,3.69,3.65,4.37,4.18,4.71,Earnings,al
8,Cost of Funding Earning Assets,0.34,0.37,0.68,0.58,0.77,Earnings,al
9,Provisions to Avg. Assets,0.03,0.00,0.13,0.13,0.11,Earnings,al


In [39]:
absolute_df = absolute_df.reset_index(drop = True)

In [42]:
absolute_df

,ECONOMIC INDICATORS,Q2-21,Q1-21,Q2-20,2020,2019,Category,state_alias
0,Past-Due and Nonaccrual Loans / Total Loans,0.80,1.00,1.00,1.11,1.63,Asset Quality,al
1,Noncurrent Loans / Total Loans,0.43,0.44,0.49,0.48,0.66,Asset Quality,al
2,Loan and Lease Allowance / Total Loans,1.33,1.34,1.23,1.31,1.23,Asset Quality,al
3,Loan and Lease Allowance / Noncurrent Loans,2.94,2.92,2.17,2.27,1.80,Asset Quality,al
4,Net Loan Losses / Total Loans,0.02,0.00,0.04,0.07,0.12,Asset Quality,al
5,Return on Assets,1.03,0.97,0.98,0.93,1.07,Earnings,al
6,Net Interest Margin,3.28,3.21,3.70,3.56,3.86,Earnings,al
7,Yield on Earning Assets,3.69,3.65,4.37,4.18,4.71,Earnings,al
8,Cost of Funding Earning Assets,0.34,0.37,0.68,0.58,0.77,Earnings,al
9,Provisions to Avg. Assets,0.03,0.00,0.13,0.13,0.11,Earnings,al


In [43]:
state_lookup = pd.read_csv('state_lookup.csv')

In [44]:
all_states = pd.merge(absolute_df, state_lookup, on="state_alias")

In [45]:
all_states

,ECONOMIC INDICATORS,Q2-21,Q1-21,Q2-20,2020,2019,Category,state_alias,state_name
0,Past-Due and Nonaccrual Loans / Total Loans,0.80,1.00,1.00,1.11,1.63,Asset Quality,al,Alabama
1,Noncurrent Loans / Total Loans,0.43,0.44,0.49,0.48,0.66,Asset Quality,al,Alabama
2,Loan and Lease Allowance / Total Loans,1.33,1.34,1.23,1.31,1.23,Asset Quality,al,Alabama
3,Loan and Lease Allowance / Noncurrent Loans,2.94,2.92,2.17,2.27,1.80,Asset Quality,al,Alabama
4,Net Loan Losses / Total Loans,0.02,0.00,0.04,0.07,0.12,Asset Quality,al,Alabama
5,Return on Assets,1.03,0.97,0.98,0.93,1.07,Earnings,al,Alabama
6,Net Interest Margin,3.28,3.21,3.70,3.56,3.86,Earnings,al,Alabama
7,Yield on Earning Assets,3.69,3.65,4.37,4.18,4.71,Earnings,al,Alabama
8,Cost of Funding Earning Assets,0.34,0.37,0.68,0.58,0.77,Earnings,al,Alabama
9,Provisions to Avg. Assets,0.03,0.00,0.13,0.13,0.11,Earnings,al,Alabama


In [47]:
all_states.to_csv("51_states_fdic.csv", index = None)